In [1]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1
!pip install rhinoMorph

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
import numpy as np
import pandas as pd
import re
import json
import rhinoMorph
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

## 데이터 불러오기

In [3]:
train_set = pd.read_csv('/content/drive/MyDrive/KTO_Project/data/PNtrain_set.csv', encoding='utf-8')
test_set = pd.read_csv('/content/drive/MyDrive/KTO_Project/data/PNtest_set.csv', encoding='utf-8')

In [4]:
train_set.head()

,review,is_pos
0,진짜 모든게 괜찮아서 더 말할 필요가 없을 정도입니다.\n너무 잘 묵다갑니다.,1
1,완전 짱 좋아용,1
2,스위트 너무너무좋아요~~!,1
3,"안녕하세요 사장님 ~ 저 금,토 2박하고 안반데기 별 보러간 놈 입니다 ^^~ 2박...",1
4,친구들과 같이 갔는데 여태 여행다녔던 숙소중 최고였다고 다들 이야기를하네요 너무 좋...,1


## 데이터 하나 특수문자 처리

In [5]:
review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", train_set['review'][0])
print(review_text)

진짜 모든게 괜찮아서 더 말할 필요가 없을 정도입니다
너무 잘 묵다갑니다


## 형태소 분석

In [6]:
rn = rhinoMorph.startRhino()

morphed_review = rhinoMorph.onlyMorph_list(rn, review_text, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'], eomi=True)
print(morphed_review)

filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!
['진짜', '모든', '괜찮다', '더', '말', '필요', '없다', '정도', '너무', '잘', '묵다', '다', '가다']


## 불용어 사전 이용하여 불용어 제거

In [7]:
stop_word = pd.read_csv('/content/drive/MyDrive/KTO_Project/data/stopword_dict.csv', encoding='utf-8')

In [8]:
stop_word = set(stop_word['stopWords'])

In [9]:
stop_word

{'\n',
 '\r\n',
 ' 위하',
 '!',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '^^',
 '_',
 '`',
 'c',
 'kg',
 'km',
 '{',
 '|',
 '}',
 '~',
 '£',
 '¥',
 '©',
 '°',
 '±',
 '·',
 '×',
 '÷',
 'ᆞ',
 '‘',
 '’',
 '₩',
 '↑',
 '∼',
 '∽',
 '─',
 '■',
 '□',
 '▲',
 '▶',
 '◀',
 '◆',
 '◇',
 '◈',
 '★',
 '☆',
 '☎',
 '♡',
 '♥',
 '♪',
 '♬',
 '。',
 '『',
 '』',
 '【',
 '】',
 '가',
 '가까스로',
 '가다',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그

In [10]:
clean_review = []
for token in morphed_review:
  if not token in stop_word:
    clean_review.append(token)
print(clean_review)

['진짜', '모든', '괜찮다', '더', '필요', '없다', '정도', '잘', '묵다']


## 데이터 전처리 함수 작성

In [19]:
def preprocessing(review, rn, stop_words=[]):
  review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)

  morphed_review = rhinoMorph.onlyMorph_list(rn, review_text, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'], eomi=True)
  
  clean_review=[]
  for token in morphed_review:
    if not token in stop_word:
      clean_review.append(token)
  return clean_review

In [20]:
clean_train_review=[]
clean_test_review=[]
for review in train_set['review']:
  clean_train_review.append(preprocessing(review, rn, stop_words=stop_word))
for review in test_set['review']:
  clean_test_review.append(preprocessing(review, rn, stop_words=stop_word))

In [21]:
clean_train_review[:2]

[['진짜', '모든', '괜찮다', '더', '필요', '없다', '정도', '잘', '묵다'], ['완전', '짱', '좋다']]

## 인덱스 벡터 변환 및 패딩

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

word_vocab = tokenizer.word_index

MAX_SEQUENCE_LENGTH = 14 # EDA 과정에서 단어 평균 개수 14개

train_inputs = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 데이터 벡터화
train_labels = np.array(train_set['is_pos']) # 학습 데이터 라벨
test_inputs = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') # 평가 데이터 벡터화
test_labels = np.array(test_set['is_pos']) # 평가 데이터 라벨

## 전처리된 데이터 저장

In [28]:
DATA_IN_PATH = '/content/drive/MyDrive/KTO_Project/data/'
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs = {}

data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)+1

# 전처리된 학습 데이터 넘파이 형태로 저장
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)
# 전처리된 평가 데이터 넘파이 형태로 저장
np.save(open(DATA_IN_PATH + TEST_INPUT_DATA, 'wb'), test_inputs)
np.save(open(DATA_IN_PATH + TEST_LABEL_DATA, 'wb'), test_labels)

# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)